In [1]:
import pandas as pd
import datetime
import os
import numpy as np
import json
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Planner_request/Faith_zips_for_LifeOccasionMoments_20190820'

In [2]:
store_list=pd.read_excel("./BL_Life'sOccasions_trafficdriver_12 Zips 6.21.2019.xlsx",sheetname="zips_by_store",dtype=str)
df_output=store_list[:]
del df_output['zip']
del df_output['revenue_flag']
del df_output['dist_miles']
df_output=df_output.drop_duplicates()
store_list=store_list['location_id'].unique().tolist()


In [3]:
new_ta_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_remove_8_closing_stores_JL_2019-08-20.xlsx")
print(new_ta_zips.sheet_names)
df_TA_by_store=new_ta_zips.parse("view_by_store",dtype=str)

['unique_zips_full_footprint', 'view_by_TA', 'view_by_store']


In [4]:
df_output.shape

(12, 8)

In [5]:
df_iter=df_TA_by_store[df_TA_by_store['location_id'].isin(store_list)]
df_iter=df_iter.reset_index()
del df_iter['index']

df_output_store_zip=pd.DataFrame()
for ind, row in df_iter.iterrows():
    store_number=row['location_id']
    P_zip_list=eval(row['all_trans_P_zips'])
    S_zip_list=eval(row['all_trans_S_zips'])
    
    df_P=pd.DataFrame({"zip_cd":P_zip_list},index=["P"]*len(P_zip_list)).reset_index().rename(columns={"index":"transaction_flag"})
    df_P['location_id']=store_number
    
    df_S=pd.DataFrame({"zip_cd":S_zip_list},index=["S"]*len(S_zip_list)).reset_index().rename(columns={"index":"transaction_flag"})
    df_S['location_id']=store_number
    
    
    df_output_store_zip=df_output_store_zip.append(df_P)
    df_output_store_zip=df_output_store_zip.append(df_S)
print(df_output_store_zip.shape)
print(df_output_store_zip['location_id'].nunique())
df_output=pd.merge(df_output,df_output_store_zip,on="location_id",how="left")

(170, 3)
12


In [6]:
df_output.head(2)

,location_id,store_zip,latitude_meas,longitude_meas,address_line_1,address_line_2,city_nm,state_nm,transaction_flag,zip_cd
0,153,45030,39.25928,-84.803908,10939 NEW HAVEN RD,nan,HARRISON,OH,P,45030
1,153,45030,39.25928,-84.803908,10939 NEW HAVEN RD,nan,HARRISON,OH,P,47060


In [7]:
df_output['latitude_meas']=df_output['latitude_meas'].astype(float)
df_output['longitude_meas']=df_output['longitude_meas'].astype(float)
df_output=df_output.reset_index()
del df_output['index']
df_output['dist_miles']=np.nan
for ind, row in df_output.iterrows():
    zip_cd=row['zip_cd']
    try:
        dist=haversine((row['latitude_meas'],row['longitude_meas']),zip_centers[zip_cd],miles=True)
    except:
        dist=np.nan
        
    df_output.loc[ind,"dist_miles"]=dist
    
        
        

In [8]:
df_output=df_output[((df_output['transaction_flag']=="P") | (df_output['dist_miles']<=20))]

In [9]:
df_unique_zip=df_output[['zip_cd','transaction_flag']].drop_duplicates()
df_unique_zip=df_unique_zip.sort_values(["zip_cd","transaction_flag"]).drop_duplicates("zip_cd")
df_unique_zip=df_unique_zip.sort_values("transaction_flag")
df_unique_zip.shape

(90, 2)

In [10]:
writer=pd.ExcelWriter("./BL_Life'sOccasions_trafficdriver_12 Zips 6.21.2019_JL_2019-08-20.xlsx",engine="xlsxwriter")
df_unique_zip.to_excel(writer,"unique_zips_full_footprint",index=False)
df_output.to_excel(writer,"zips_by_store",index=False)
writer.save()